<a href="https://colab.research.google.com/github/emello23/Alura-Gemini/blob/main/Aprendizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



"""
Script para extrair texto de um PDF, gerar resumo e tópicos com a API do Gemini
e criar um chatbot para interagir com o conteúdo.
"""

In [11]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Instalar/Atualizar as bibliotecas necessárias
!pip install --upgrade google-genai PyMuPDF


In [13]:
import os
from google.colab import userdata
from google import genai
from google.genai import types # Importação ativa
import fitz  # Importa a biblioteca PyMuPDF

In [14]:
# Importa classes do módulo IPython.display para formatar a saída no notebook.
# - HTML: Permite exibir conteúdo HTML diretamente.
# - Markdown: Permite exibir conteúdo formatado em Markdown.

from IPython.display import HTML, Markdown

In [15]:
# Configurar a chave da API
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()

In [16]:
# Definir o modelo a ser usado
modelo = "gemini-2.0-flash"


In [17]:
# --- PARTE ONDE VOCÊ CARREGA O PDF ---
# Atualize o caminho do PDF para apontar para o seu arquivo no Google Drive
# Alternativa: Coloque o arquivo na sua pasta raiz do My Drive
#              e renomei-o para aprendizado.pdf mantendo este mesmo caminho.
caminho_do_pdf = "/content/drive/My Drive/aprendizado.pdf" # <-- ATUALIZE ESTA LINHA COM O CAMINHO CORRETO'

In [18]:
def extrair_texto_de_pdf(caminho_do_arquivo):
    """Extrai texto de um arquivo PDF."""
    texto_completo = ""
    try:
        documento = fitz.open(caminho_do_arquivo)
        for pagina_num in range(documento.page_count):
            pagina = documento.load_page(pagina_num)
            texto_completo += pagina.get_text()
        documento.close()
    except Exception as e:
        print(f"Erro ao extrair texto do PDF: {e}")
        texto_completo = None # Retorna None em caso de erro
    return texto_completo



In [19]:
texto_do_pdf = extrair_texto_de_pdf(caminho_do_pdf)



In [20]:
if texto_do_pdf: # Verifica se a extração do texto foi bem-sucedida
    # --- Alteração aqui para imprimir em Markdown ---
    display(Markdown("## Texto extraído do PDF (primeiros 500 caracteres):"))
    print(texto_do_pdf[:500] + "...") # Este print pode permanecer, pois é um trecho longo
    # --- Fim da alteração ---

    # --- USANDO GEMINI PARA AJUDAR COM RESUMO E TÓPICOS ---
    # Gerar um resumo inicial
    try:
        response_resumo = client.models.generate_content(
            model=modelo,
            contents=f"Por favor, gere um resumo conciso do seguinte texto:\n\n{texto_do_pdf}"
        )
        if response_resumo and response_resumo.text: # Verifica se a resposta e o texto existem
            resumo = response_resumo.text
            # --- Alteração aqui para imprimir em Markdown ---
            display(Markdown("## Resumo gerado pelo Gemini:"))
            display(Markdown(resumo))
            # --- Fim da alteração ---
        else:
            # --- Alteração aqui para imprimir em Markdown ---
            display(Markdown("## Não foi possível gerar um resumo."))
            # --- Fim da alteração ---
    except Exception as e:
        print(f"Erro ao gerar resumo: {e}")


    # Tentar identificar tópicos principais
    try:
        response_topicos = client.models.generate_content(
            model=modelo,
            contents=f"Por favor, liste os principais tópicos e sub-tópicos (numerados se possível) encontrados no seguinte texto:\n\n{texto_do_pdf}"
        )
        if response_topicos and response_topicos.text: # Verifica se a resposta e o texto existem
            topicos_identificados = response_topicos.text
            # --- Alteração aqui para imprimir em Markdown ---
            display(Markdown("## Tópicos identificados pelo Gemini:"))
            display(Markdown(topicos_identificados))
            # --- Fim da alteração ---
        else:
             # --- Alteração aqui para imprimir em Markdown ---
            display(Markdown("## Não foi possível identificar tópicos."))
            # --- Fim da alteração ---
    except Exception as e:
        print(f"Erro ao identificar tópicos: {e}")

    # --- PARTE DO CHATBOT ---
    # --- Alterações aqui para imprimir em Markdown ---
    display(Markdown("## Ok! Vamos aprender sobre o conteúdo do documento."))
    display(Markdown("Eu farei perguntas sobre o conteúdo do documento. Tente respondê-las!"))
    display(Markdown("Digite 'fim' para encerrar o chat."))
    display(Markdown("Desconsidere a 'Pergunta 1:' digitando 'OK'"))
    display(Markdown("### Vamos começar!"))
    # --- Fim das alterações ---

    # Abordagem para gerar perguntas
    try:
        response_perguntas = client.models.generate_content(
            model=modelo,
            contents=f"Com base no seguinte texto, gere 10 perguntas sobre o seu conteúdo para que um usuário possa respondê-las. Liste as perguntas numeradas.\n\n{texto_do_pdf}"
        )

        if response_perguntas and response_perguntas.text: # Verifica se a resposta e o texto existem
            perguntas_geradas = response_perguntas.text.split('\n') # Divide as perguntas em uma lista
            # Filtra linhas vazias e numerações que não são perguntas válidas
            perguntas_validas = [p.strip() for p in perguntas_geradas if p.strip() and any(char.isdigit() for char in p.strip().split('.')[0])]


            if not perguntas_validas:
                 # --- Alteração aqui para imprimir em Markdown ---
                display(Markdown("## Não foi possível gerar perguntas a partir do PDF."))
                 # --- Fim da alteração ---
            else:
                indice_pergunta_atual = 0

                while indice_pergunta_atual < len(perguntas_validas):
                    pergunta_atual = perguntas_validas[indice_pergunta_atual]
                     # --- Alteração aqui para imprimir em Markdown ---
                    display(Markdown(f"### Pergunta {indice_pergunta_atual + 1}: {pergunta_atual}"))
                    # --- Fim da alteração ---

                    resposta_usuario = input("Sua resposta: ")

                    if resposta_usuario.lower() == "fim":
                        print("Chat encerrado. Até a próxima!") # Este print pode permanecer
                        break

                    # Usar o Gemini para avaliar a resposta do usuário
                    try:
                        chat_avaliacao_config = types.GenerateContentConfig(
                          system_instruction = f"Você é um avaliador de respostas baseado no seguinte texto. A pergunta foi: '{pergunta_atual}'. Avalie a resposta do usuário ('{resposta_usuario}') com base no texto original e diga se ela está correta, incorreta ou parcialmente correta, explicando brevemente por que. O texto original é:\n\n{texto_do_pdf}"
                        )
                        chat_avaliacao = client.chats.create(model=modelo, config=chat_avaliacao_config)
                        response_avaliacao = chat_avaliacao.send_message(f"Pergunta: {pergunta_atual}\nResposta do usuário: {resposta_usuario}")

                        if response_avaliacao and response_avaliacao.text: # Verifica se a resposta e o texto existem
                             # --- Alteração aqui para imprimir em Markdown ---
                            display(Markdown("### Avaliação do Gemini: "))
                            display(Markdown(response_avaliacao.text))
                             # --- Fim da alteração ---
                        else:
                             # --- Alteração aqui para imprimir em Markdown ---
                            display(Markdown("## Não foi possível avaliar a resposta."))
                            # --- Fim da alteração ---

                    except Exception as e:
                        print(f"Erro ao avaliar a resposta: {e}")

                    indice_pergunta_atual += 1 # Passa para a próxima pergunta

                if indice_pergunta_atual == len(perguntas_validas):
                   # --- Alteração aqui para imprimir em Markdown ---
                   display(Markdown("\n## Você respondeu a todas as perguntas. Chat encerrado. Até a próxima!"))
                   # --- Fim da alteração ---
        else:
             # --- Alteração aqui para imprimir em Markdown ---
            display(Markdown("## Não foi possível gerar perguntas a partir do PDF."))
            # --- Fim da alteração ---
    except Exception as e:
        print(f"Erro ao gerar perguntas ou executar o loop do chatbot: {e}")

else:
     # --- Alteração aqui para imprimir em Markdown ---
    display(Markdown("## Não foi possível prosseguir porque a extração do texto do PDF falhou."))
    # --- Fim da alteração ---

## Texto extraído do PDF (primeiros 500 caracteres):

Aula2
Lenalda Andrade Santos
A REVOLUÇÃO FRANCESA
META
Revisar e aprofundar o estudo do movimento que se transformou no modelo clássico de 
revolução burguesa. 
OBJETIVOS
Ao ﬁ nal desta aula, o aluno deverá:
Analisar a Revolução Francesa como um evento capital de toda a 
história do mundo moderno.
Identiﬁ car a contribuição do movimento revolucionário francês para o ﬁ m da
 antiga ordem feudal-absolutista.
Indicar bibliograﬁ a que permita aos alunos aprofundarem o estudo do tema 
das revoluções ...


## Resumo gerado pelo Gemini:

Este texto é uma revisão aprofundada da Revolução Francesa, abordando sua importância como modelo de revolução burguesa e seu impacto no fim do feudalismo-absolutismo. Ele analisa a França pré-revolucionária, destacando o poder crescente da burguesia em contraste com os privilégios da nobreza e do clero, e a insatisfação do Terceiro Estado. O texto explora a convocação dos Estados Gerais, a tomada da Bastilha, as fases da revolução (desde a inspiração parlamentar inglesa até a ditadura jacobina e o Terror), e a ascensão de Napoleão. As contribuições da Revolução são enumeradas, incluindo a liquidação do feudalismo, a afirmação da igualdade de direitos, a destruição do direito divino dos governantes e a separação entre Estado e Igreja. O texto também discute o papel da burguesia no movimento, o significado da revolução e as lutas contra o Antigo Regime.


## Tópicos identificados pelo Gemini:

...

Sua resposta: Iniciou a derrota da nobreza


### Avaliação do Gemini: 

A resposta do usuário está parcialmente correta. A Tomada da Bastilha simbolizou um golpe contra o absolutismo monárquico e, por extensão, contra a nobreza que o sustentava. No entanto, a resposta simplifica o significado complexo do evento e sua importância como marco inicial da Revolução Francesa. A tomada da Bastilha também representou a entrada das massas populares no processo revolucionário, marcando uma escalada no confronto com o Antigo Regime.

### Pergunta 9: 8.  Como a palavra "revolução" mudou de significado com a Queda da Bastilha, segundo a análise de Hannah Arendt?

Sua resposta: fim
Chat encerrado. Até a próxima!
